<a href="https://colab.research.google.com/github/jaymoundekar18/CourierCompanyChargeBillAnalysis/blob/main/Courier%20Company%20Charged%20Bill%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Required Libraries

In [1]:
import math
import numpy as np
import pandas as pd

### Reading all the excel files

In [2]:
pincode = pd.read_excel(r"Company X - Pincode Zones.xlsx")
order = pd.read_excel(r"Company X - Order Report.xlsx")
sku = pd.read_excel(r"Company X - SKU Master.xlsx")
invoice = pd.read_excel(r"Courier Company - Invoice.xlsx")
rates = pd.read_excel(r"Courier Company - Rates.xlsx")

### Getting a glance of all the files

In [3]:
pincode.head()

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [4]:
order.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [5]:
sku.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [6]:
invoice.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [7]:
rates.head()

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.50,29.5,23.6,13.6,23.6
1,B,1.00,33.0,28.3,20.5,28.3
2,C,1.25,40.1,38.9,31.9,38.9
3,D,1.50,45.4,44.8,41.3,44.8
4,E,2.00,56.6,55.5,50.7,55.5


### Getting the column names of all the files

In [8]:
pincode.columns

Index(['Warehouse Pincode', 'Customer Pincode', 'Zone'], dtype='object')

In [9]:
order.columns

Index(['ExternOrderNo', 'SKU', 'Order Qty'], dtype='object')

In [10]:
sku.columns

Index(['SKU', 'Weight (g)'], dtype='object')

In [11]:
invoice.columns

Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode',
       'Customer Pincode', 'Zone', 'Type of Shipment', 'Billing Amount (Rs.)'],
      dtype='object')

In [12]:
rates.columns

Index(['Zone', 'Weight Slabs', 'Forward Fixed Charge',
       'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
       'RTO Additional Weight Slab Charge'],
      dtype='object')

### Merging the data to get the total weight of particular Order_ID

In [13]:
merged_data = pd.merge(order, sku, on='SKU', how='left')

merged_data['Total Weight (kg)'] = (merged_data['Weight (g)'] * merged_data['Order Qty']) / 1000

In [14]:
merged_data

,ExternOrderNo,SKU,Order Qty,Weight (g),Total Weight (kg)
0,2001827036,8904223818706,1.0,127,0.127
1,2001827036,8904223819093,1.0,150,0.150
2,2001827036,8904223819109,1.0,100,0.100
3,2001827036,8904223818430,1.0,165,0.165
4,2001827036,8904223819277,1.0,350,0.350
...,...,...,...,...,...
396,2001806229,8904223818942,1.0,133,0.133
397,2001806229,8904223818850,1.0,240,0.240
398,2001806226,8904223818850,2.0,240,0.480
399,2001806210,8904223816214,1.0,120,0.120


In [15]:
merged_data.columns

Index(['ExternOrderNo', 'SKU', 'Order Qty', 'Weight (g)', 'Total Weight (kg)'], dtype='object')

### Calculating Total weights

In [16]:
result = merged_data.groupby('ExternOrderNo')['Total Weight (kg)'].sum().reset_index()
result

,ExternOrderNo,Total Weight (kg)
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245
...,...,...
119,2001821995,0.477
120,2001822466,1.376
121,2001823564,0.672
122,2001825261,1.557


### Creating a new dataframe

In [17]:
final_data = pd.merge(result, invoice, left_on='ExternOrderNo', right_on='Order ID', how='left')

In [18]:
final_data.head()

,ExternOrderNo,Total Weight (kg),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,0.220,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,0.480,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,0.500,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1.302,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
4,2001806233,0.245,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3


In [19]:
final_data.columns

Index(['ExternOrderNo', 'Total Weight (kg)', 'AWB Code', 'Order ID',
       'Charged Weight', 'Warehouse Pincode', 'Customer Pincode', 'Zone',
       'Type of Shipment', 'Billing Amount (Rs.)'],
      dtype='object')

### Removed the unwanted column and rearranged them 

In [20]:
final_data = final_data[['Order ID', 'AWB Code', 'Total Weight (kg)', 'Charged Weight', 'Zone', 'Type of Shipment', 'Billing Amount (Rs.)']]

In [21]:
final_data

,Order ID,AWB Code,Total Weight (kg),Charged Weight,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,1091117221940,0.220,2.92,b,Forward charges,174.5
1,2001806226,1091117222065,0.480,0.68,d,Forward charges,90.2
2,2001806229,1091117222080,0.500,0.71,d,Forward charges,90.2
3,2001806232,1091117222124,1.302,1.30,d,Forward charges,135.0
4,2001806233,1091117222135,0.245,0.78,b,Forward charges,61.3
...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.50,d,Forward charges,45.4
120,2001822466,1091121305541,1.376,1.10,d,Forward charges,135.0
121,2001823564,1091121666133,0.672,0.70,d,Forward and RTO charges,172.8
122,2001825261,1091121981575,1.557,1.60,d,Forward and RTO charges,345.0


In [22]:
final_data.columns

Index(['Order ID', 'AWB Code', 'Total Weight (kg)', 'Charged Weight', 'Zone',
       'Type of Shipment', 'Billing Amount (Rs.)'],
      dtype='object')

### Calculating weight slab as per X

In [23]:
final_data['Weight slab as per X (KG)'] = final_data['Total Weight (kg)'].apply(lambda x: (math.ceil(x*2)/2) )

In [24]:
final_data

,Order ID,AWB Code,Total Weight (kg),Charged Weight,Zone,Type of Shipment,Billing Amount (Rs.),Weight slab as per X (KG)
0,2001806210,1091117221940,0.220,2.92,b,Forward charges,174.5,0.5
1,2001806226,1091117222065,0.480,0.68,d,Forward charges,90.2,0.5
2,2001806229,1091117222080,0.500,0.71,d,Forward charges,90.2,0.5
3,2001806232,1091117222124,1.302,1.30,d,Forward charges,135.0,1.5
4,2001806233,1091117222135,0.245,0.78,b,Forward charges,61.3,0.5
...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.50,d,Forward charges,45.4,0.5
120,2001822466,1091121305541,1.376,1.10,d,Forward charges,135.0,1.5
121,2001823564,1091121666133,0.672,0.70,d,Forward and RTO charges,172.8,1.0
122,2001825261,1091121981575,1.557,1.60,d,Forward and RTO charges,345.0,2.0


### Converting the Zone column and adding the weight slab as per courier company

In [25]:
final_data['Zone'] = final_data['Zone'].map({'a': 'A', 'b': 'B', 'c': 'C', 'd': 'D', 'e': 'E'})

In [26]:
final_data

,Order ID,AWB Code,Total Weight (kg),Charged Weight,Zone,Type of Shipment,Billing Amount (Rs.),Weight slab as per X (KG)
0,2001806210,1091117221940,0.220,2.92,B,Forward charges,174.5,0.5
1,2001806226,1091117222065,0.480,0.68,D,Forward charges,90.2,0.5
2,2001806229,1091117222080,0.500,0.71,D,Forward charges,90.2,0.5
3,2001806232,1091117222124,1.302,1.30,D,Forward charges,135.0,1.5
4,2001806233,1091117222135,0.245,0.78,B,Forward charges,61.3,0.5
...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.50,D,Forward charges,45.4,0.5
120,2001822466,1091121305541,1.376,1.10,D,Forward charges,135.0,1.5
121,2001823564,1091121666133,0.672,0.70,D,Forward and RTO charges,172.8,1.0
122,2001825261,1091121981575,1.557,1.60,D,Forward and RTO charges,345.0,2.0


In [27]:
fdata = pd.merge(final_data, rates, on='Zone', how='left')

In [28]:
fdata.head()

,Order ID,AWB Code,Total Weight (kg),Charged Weight,Zone,Type of Shipment,Billing Amount (Rs.),Weight slab as per X (KG),Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,2001806210,1091117221940,0.220,2.92,B,Forward charges,174.5,0.5,1.0,33.0,28.3,20.5,28.3
1,2001806226,1091117222065,0.480,0.68,D,Forward charges,90.2,0.5,1.5,45.4,44.8,41.3,44.8
2,2001806229,1091117222080,0.500,0.71,D,Forward charges,90.2,0.5,1.5,45.4,44.8,41.3,44.8
3,2001806232,1091117222124,1.302,1.30,D,Forward charges,135.0,1.5,1.5,45.4,44.8,41.3,44.8
4,2001806233,1091117222135,0.245,0.78,B,Forward charges,61.3,0.5,1.0,33.0,28.3,20.5,28.3


In [29]:
fdata.columns

Index(['Order ID', 'AWB Code', 'Total Weight (kg)', 'Charged Weight', 'Zone',
       'Type of Shipment', 'Billing Amount (Rs.)', 'Weight slab as per X (KG)',
       'Weight Slabs', 'Forward Fixed Charge',
       'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
       'RTO Additional Weight Slab Charge'],
      dtype='object')

In [30]:
fdata = fdata.drop(['Forward Fixed Charge', 'Forward Additional Weight Slab Charge', 'RTO Fixed Charge', 'RTO Additional Weight Slab Charge'], axis=1)


In [31]:
fdata.head(1)

,Order ID,AWB Code,Total Weight (kg),Charged Weight,Zone,Type of Shipment,Billing Amount (Rs.),Weight slab as per X (KG),Weight Slabs
0,2001806210,1091117221940,0.22,2.92,B,Forward charges,174.5,0.5,1.0


### Renaming the column and rearranging them

In [32]:
fdata['Delivery Zone charged by Courier Company'] = fdata[['Zone']]
fdata = fdata.rename(columns={'Total Weight (kg)': 'Total weight as per X (KG)','Charged Weight':'Total weight as per Courier Company (KG)','Zone':'Delivery Zone as per X','Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)','Weight Slabs':'Weight slab charged by Courier Company (KG)'})
fdata.head()

,Order ID,AWB Code,Total weight as per X (KG),Total weight as per Courier Company (KG),Delivery Zone as per X,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone charged by Courier Company
0,2001806210,1091117221940,0.220,2.92,B,Forward charges,174.5,0.5,1.0,B
1,2001806226,1091117222065,0.480,0.68,D,Forward charges,90.2,0.5,1.5,D
2,2001806229,1091117222080,0.500,0.71,D,Forward charges,90.2,0.5,1.5,D
3,2001806232,1091117222124,1.302,1.30,D,Forward charges,135.0,1.5,1.5,D
4,2001806233,1091117222135,0.245,0.78,B,Forward charges,61.3,0.5,1.0,B


In [33]:
fdata = fdata[['Order ID', 'AWB Code', 'Total weight as per X (KG)', 'Weight slab as per X (KG)','Total weight as per Courier Company (KG)','Weight slab charged by Courier Company (KG)',  'Delivery Zone as per X', 'Delivery Zone charged by Courier Company', 'Type of Shipment', 'Charges Billed by Courier Company (Rs.)']]
fdata.head()

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,1.0,B,B,Forward charges,174.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.5,D,D,Forward charges,90.2
2,2001806229,1091117222080,0.500,0.5,0.71,1.5,D,D,Forward charges,90.2
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,Forward charges,135.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B,Forward charges,61.3


### Calculating the Expected Charges as per X

In [34]:
rates.columns

Index(['Zone', 'Weight Slabs', 'Forward Fixed Charge',
       'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
       'RTO Additional Weight Slab Charge'],
      dtype='object')

In [35]:
rates

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.50,29.5,23.6,13.6,23.6
1,B,1.00,33.0,28.3,20.5,28.3
2,C,1.25,40.1,38.9,31.9,38.9
3,D,1.50,45.4,44.8,41.3,44.8
4,E,2.00,56.6,55.5,50.7,55.5


In [36]:
res = []
for i,j,k in zip(fdata['Type of Shipment'],fdata['Delivery Zone as per X'],fdata['Weight slab as per X (KG)']):
    if i == 'Forward charges':
        if k > 0.5:
            res.append(round((rates.loc[rates['Zone'] == j, 'Forward Fixed Charge'].values[0]) + ((k-0.5)/0.5 * (rates.loc[rates['Zone'] == j, 'Forward Additional Weight Slab Charge'].values[0])),1))
        else:
            res.append(round(rates.loc[rates['Zone'] == j, 'Forward Fixed Charge'].values[0]))
    if i == 'Forward and RTO charges':
        if k> 0.5:
            res.append(round((rates.loc[rates['Zone'] == j, 'RTO Fixed Charge'].values[0]) + ((k-0.5)/0.5 * (rates.loc[rates['Zone'] == j, 'RTO Additional Weight Slab Charge'].values[0])),1))
        else:
            res.append(round(rates['RTO Fixed Charge'][rates['Zone']==j]))

In [37]:
expectedCharges = pd.DataFrame(res)
expectedCharges.columns = ['Expected Charge as per X (Rs.)']
expectedCharges

,Expected Charge as per X (Rs.)
0,33
1,45
2,45
3,135.0
4,33
...,...
119,45
120,135.0
121,86.1
122,175.7


In [38]:
expectedCharges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 1 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Expected Charge as per X (Rs.)  124 non-null    object
dtypes: object(1)
memory usage: 1.1+ KB


In [39]:
expectedCharges['Expected Charge as per X (Rs.)'] = pd.to_numeric(expectedCharges['Expected Charge as per X (Rs.)'], errors='coerce')
expectedCharges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 1 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Expected Charge as per X (Rs.)  121 non-null    float64
dtypes: float64(1)
memory usage: 1.1 KB


In [40]:
fdata.columns

Index(['Order ID', 'AWB Code', 'Total weight as per X (KG)',
       'Weight slab as per X (KG)', 'Total weight as per Courier Company (KG)',
       'Weight slab charged by Courier Company (KG)', 'Delivery Zone as per X',
       'Delivery Zone charged by Courier Company', 'Type of Shipment',
       'Charges Billed by Courier Company (Rs.)'],
      dtype='object')

In [41]:
fdata.head()

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,1.0,B,B,Forward charges,174.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.5,D,D,Forward charges,90.2
2,2001806229,1091117222080,0.500,0.5,0.71,1.5,D,D,Forward charges,90.2
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,Forward charges,135.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B,Forward charges,61.3


### Concatenating the expected charges

In [42]:
Result = pd.concat([fdata,expectedCharges],axis=1)

In [43]:
Result

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Expected Charge as per X (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,1.0,B,B,Forward charges,174.5,33.0
1,2001806226,1091117222065,0.480,0.5,0.68,1.5,D,D,Forward charges,90.2,45.0
2,2001806229,1091117222080,0.500,0.5,0.71,1.5,D,D,Forward charges,90.2,45.0
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,Forward charges,135.0,135.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B,Forward charges,61.3,33.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,1.5,D,D,Forward charges,45.4,45.0
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,D,D,Forward charges,135.0,135.0
121,2001823564,1091121666133,0.672,1.0,0.70,1.5,D,D,Forward and RTO charges,172.8,86.1
122,2001825261,1091121981575,1.557,2.0,1.60,1.5,D,D,Forward and RTO charges,345.0,175.7


### Calculating the Difference Between Expected Charges and Billed Charges

In [44]:
Result['Difference Between Expected Charges and Billed Charges (Rs.)'] = Result['Expected Charge as per X (Rs.)']-Result['Charges Billed by Courier Company (Rs.)']

In [45]:
Result

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Expected Charge as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,1.0,B,B,Forward charges,174.5,33.0,-141.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.5,D,D,Forward charges,90.2,45.0,-45.2
2,2001806229,1091117222080,0.500,0.5,0.71,1.5,D,D,Forward charges,90.2,45.0,-45.2
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,Forward charges,135.0,135.0,0.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B,Forward charges,61.3,33.0,-28.3
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,1.5,D,D,Forward charges,45.4,45.0,-0.4
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,D,D,Forward charges,135.0,135.0,0.0
121,2001823564,1091121666133,0.672,1.0,0.70,1.5,D,D,Forward and RTO charges,172.8,86.1,-86.7
122,2001825261,1091121981575,1.557,2.0,1.60,1.5,D,D,Forward and RTO charges,345.0,175.7,-169.3


In [46]:
Result = Result.drop(['Type of Shipment'], axis=1)

### Final Output 1 : "Order level calculation"

In [47]:
Result

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Expected Charge as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,1.0,B,B,174.5,33.0,-141.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.5,D,D,90.2,45.0,-45.2
2,2001806229,1091117222080,0.500,0.5,0.71,1.5,D,D,90.2,45.0,-45.2
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,135.0,135.0,0.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B,61.3,33.0,-28.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,1.5,D,D,45.4,45.0,-0.4
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,D,D,135.0,135.0,0.0
121,2001823564,1091121666133,0.672,1.0,0.70,1.5,D,D,172.8,86.1,-86.7
122,2001825261,1091121981575,1.557,2.0,1.60,1.5,D,D,345.0,175.7,-169.3


### Creating csv file of the output

In [48]:
Result.to_csv("Final_Result_Output1.csv", index=False)

In [49]:
Result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124 entries, 0 to 123
Data columns (total 11 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Order ID                                                      124 non-null    int64  
 1   AWB Code                                                      124 non-null    int64  
 2   Total weight as per X (KG)                                    124 non-null    float64
 3   Weight slab as per X (KG)                                     124 non-null    float64
 4   Total weight as per Courier Company (KG)                      124 non-null    float64
 5   Weight slab charged by Courier Company (KG)                   124 non-null    float64
 6   Delivery Zone as per X                                        124 non-null    object 
 7   Delivery Zone charged by Courier Company                      124 non-n

### Summary table

#### Count of "Correctly Charged" "Over Charged" "Under Charged"

In [50]:
correctCount = Result['Difference Between Expected Charges and Billed Charges (Rs.)'][Result['Difference Between Expected Charges and Billed Charges (Rs.)']==0].count()

correctCount

35

In [51]:
underCount = Result['Difference Between Expected Charges and Billed Charges (Rs.)'][Result['Difference Between Expected Charges and Billed Charges (Rs.)']<0].count()

underCount

77

In [52]:
overCount = Result['Difference Between Expected Charges and Billed Charges (Rs.)'][Result['Difference Between Expected Charges and Billed Charges (Rs.)']>0].count()

overCount

9

In [53]:
correctCountCharge = Result['Charges Billed by Courier Company (Rs.)'][Result['Difference Between Expected Charges and Billed Charges (Rs.)']==0].sum()

correctCountCharge

3568.2999999999993

In [54]:
underCountCharge = Result['Difference Between Expected Charges and Billed Charges (Rs.)'][Result['Difference Between Expected Charges and Billed Charges (Rs.)']<0].sum()

underCountCharge

-4253.8

In [55]:
overCountCharge = Result['Difference Between Expected Charges and Billed Charges (Rs.)'][Result['Difference Between Expected Charges and Billed Charges (Rs.)']>0].sum()

overCountCharge

337.19999999999993

In [56]:
condition = ['Total orders where X has been correctly charged','Total Orders where X has been overcharged','Total Orders where X has been undercharged']

In [57]:
count = [correctCount,overCount,underCount]

In [58]:
amount = [correctCountCharge,overCountCharge,underCountCharge]

In [59]:
Summary = pd.DataFrame({'':condition,'Count':count,'Amount (Rs.)':amount})

In [60]:
Summary

,,Count,Amount (Rs.)
0,Total orders where X has been correctly charged,35,3568.3
1,Total Orders where X has been overcharged,9,337.2
2,Total Orders where X has been undercharged,77,-4253.8


### Creating csv file of output 2

In [61]:
Summary.to_csv("Final_Result_Output2.csv", index=False)